In [3]:
class PCFG:
    def __init__(self):
        self.grammar = {}

    def add_production(self, productions):
        for non_terminal, production_list in productions.items():
            if non_terminal not in self.grammar:
                self.grammar[non_terminal] = []
            for production, probability in production_list:
                self.grammar[non_terminal].append((production, probability))

    def inside_algorithm(self, sentence):
        n = len(sentence)
        chart = [[{} for _ in range(n+1)] for _ in range(n+1)]

        # Initialize chart for single-word constituents
        for i in range(1, n+1):
            for non_terminal, productions in self.grammar.items():
                for production, probability in productions:
                    if len(production) == 1 and production[0] == sentence[i-1]:
                        chart[i][i][non_terminal] = probability

        # Fill chart for longer constituents
        for span in range(2, n+1):
            for begin in range(1, n-span+2):
                end = begin + span - 1
                for split in range(begin, end):
                    for A, B_list in self.grammar.items():
                        for B, B_prob in B_list:
                            for C, C_prob in self.grammar.get(B, []):
                                prob = B_prob * C_prob
                                if chart[begin][split].get(B) is not None and chart[split+1][end].get(C) is not None:
                                    if A not in chart[begin][end]:
                                        chart[begin][end][A] = 0.0
                                    chart[begin][end][A] += prob * chart[begin][split][B] * chart[split+1][end][C]

        return chart

    def parse_trees(self, chart, sentence, start_symbol):
        def build_trees(begin, end, symbol):
            if begin == end:
                return [symbol]

            trees = []
            for split in range(begin, end):
                for A, B_list in self.grammar.items():
                    for B, _ in B_list:
                        for C, _ in self.grammar.get(B, []):
                            if chart[begin][split].get(B) is not None and chart[split+1][end].get(C) is not None:
                                prob = chart[begin][end][symbol] / (chart[begin][split][B] * chart[split+1][end][C])
                                if abs(chart[begin][end][symbol] - prob * chart[begin][split][B] * chart[split+1][end][C]) < 1e-6:
                                    left_trees = build_trees(begin, split, B)
                                    right_trees = build_trees(split+1, end, C)
                                    for left_tree in left_trees:
                                        for right_tree in right_trees:
                                            trees.append([A, left_tree, right_tree])
            return trees

        return build_trees(1, len(sentence), start_symbol)


# Example usage:
pcfg = PCFG()
pcfg.add_production({
    'S': [(['NP', 'VP'], 1.0)],
    'PP':[(['P','NP'],1.0)],
    'VP': [(['V', 'NP'], 0.7),(['VP', 'PP'], 0.3)],
    'P': [(['with'],1.0)],
    'V': [(['saw'],1.0)],
    'NP': [(['NP', 'PP'], 0.4),(['astronomers'],0.1),(['ears'],0.18),(['saw'],0.04),(['stars'],0.18),(['telescopes'],0.1)]
})

sentence = "astronomers saw stars with ears".split(" ")
# Compute inside probabilities
chart = pcfg.inside_algorithm(sentence)

# Print beta values (inside probabilities)
print("Beta values:")
for i in range(1, len(sentence) + 1):
    for j in range(i, len(sentence) + 1):
        for symbol, prob in chart[i][j].items():
            print(f"Beta({i},{j},{symbol}) = {prob}")

# Draw all possible parse trees
parse_trees = pcfg.parse_trees(chart, sentence, 'S')
print("\nAll possible parse trees:")
for tree in parse_trees:
    print(tree)


TypeError: unhashable type: 'list'